## MSDS 7331 - Case Study 6 - Predicting the Existence of New Particles
Daniel Crouthamel

Sophia Wu

Fabio Savorgnan

Bo Yun

In [1]:
import pandas as pd
import tensorflow as tf 
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import os

# Introduction

The intention of this study is to predict the existence of new particles using a dense neural network.

# Business Understanding

*You should always state the objective at the beginning of every case (a guideline you should follow in real life as well) and provide some initial "Business Understanding" statements (i.e., what is trying to be solved for and why might it be important)*

We received an abrupt request from our client in the superconductors industry to help predict the existence of new particles using a dense neural network. The goal should be to maximized accuracy. The client requires a write up on the design of the network, along with information to indicate that the model was trained appropriately.

# Data Evaluation / Engineering

*Summarize the data being used in the case using appropriate mediums (charts, graphs, tables); address questions such as: Are there missing values? Which variables are needed (which ones are not)? What assumptions or conclusions are you drawing that need to be relayed to your audience?*


In [2]:
# Load Data
df = pd.read_csv('data/all_train.csv.gz')

In [3]:
# Load Dataset and profile it, only need to do once - so comment out.
# from pandas_profiling import ProfileReport
# profile = ProfileReport(df, title="New Particles EDA", minimal=True)
# profile.to_file(output_file="NewParticlesEDA.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 504.12it/s]


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000000 entries, 0 to 6999999
Data columns (total 29 columns):
 #   Column   Dtype  
---  ------   -----  
 0   # label  float64
 1   f0       float64
 2   f1       float64
 3   f2       float64
 4   f3       float64
 5   f4       float64
 6   f5       float64
 7   f6       float64
 8   f7       float64
 9   f8       float64
 10  f9       float64
 11  f10      float64
 12  f11      float64
 13  f12      float64
 14  f13      float64
 15  f14      float64
 16  f15      float64
 17  f16      float64
 18  f17      float64
 19  f18      float64
 20  f19      float64
 21  f20      float64
 22  f21      float64
 23  f22      float64
 24  f23      float64
 25  f24      float64
 26  f25      float64
 27  f26      float64
 28  mass     float64
dtypes: float64(29)
memory usage: 1.5 GB


In [4]:
df.describe()

,# label,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f18,f19,f20,f21,f22,f23,f24,f25,f26,mass
count,7.000000e+06,7.000000e+06,7.000000e+06,7.000000e+06,7.000000e+06,7.000000e+06,7.000000e+06,7.000000e+06,7.000000e+06,7.000000e+06,...,7.000000e+06,7.000000e+06,7.000000e+06,7.000000e+06,7.000000e+06,7.000000e+06,7.000000e+06,7.000000e+06,7.000000e+06,7.000000e+06
mean,5.001256e-01,1.612528e-02,4.770022e-04,2.686578e-05,1.056081e-02,-1.050026e-04,2.765919e-03,1.815953e-02,2.510948e-05,4.345870e-04,...,1.164789e-02,-1.127097e-04,7.686731e-05,2.909202e-04,1.228774e-02,9.778378e-03,5.269844e-03,-1.760961e-03,1.533136e-02,1.000107e+03
std,5.000000e-01,1.004417e+00,9.974864e-01,1.000080e+00,9.956003e-01,9.998670e-01,1.000957e+00,9.867746e-01,9.965867e-01,1.000007e+00,...,1.002725e+00,1.000038e+00,1.000033e+00,1.000170e+00,1.010477e+00,1.005418e+00,1.009990e+00,9.844511e-01,9.822799e-01,3.534255e+02
min,0.000000e+00,-1.960549e+00,-2.365355e+00,-1.732165e+00,-9.980274e+00,-1.732137e+00,-1.054221e+00,-3.034787e+00,-2.757853e+00,-1.732359e+00,...,-1.728284e+00,-2.281867e+00,-1.731758e+00,-5.736825e-01,-3.631608e+00,-4.729473e+00,-2.062223e+01,-3.452634e+00,-2.632761e+00,5.000000e+02
25%,0.000000e+00,-7.288206e-01,-7.332548e-01,-8.656704e-01,-6.092291e-01,-8.658025e-01,-1.054221e+00,-7.566092e-01,-7.014146e-01,-8.656543e-01,...,-7.423630e-01,-7.206846e-01,-8.656855e-01,-5.736825e-01,-5.417942e-01,-5.115522e-01,-3.543870e-01,-6.925097e-01,-7.943804e-01,7.500000e+02
50%,1.000000e+00,-3.930319e-02,8.523957e-04,3.199154e-04,1.963316e-02,-5.070131e-04,-5.983562e-03,-1.499527e-01,-1.067553e-04,1.384781e-03,...,-8.992496e-02,-6.735953e-05,-4.424527e-04,-5.736825e-01,-1.602760e-01,-3.144032e-01,-3.265228e-01,-3.570301e-01,-8.828640e-02,1.000000e+03
75%,1.000000e+00,6.900799e-01,7.347832e-01,8.659464e-01,6.798818e-01,8.657646e-01,8.504885e-01,7.686690e-01,7.013194e-01,8.665976e-01,...,6.423185e-01,7.204921e-01,8.659566e-01,-5.736825e-01,4.812194e-01,1.634892e-01,-2.337671e-01,4.753128e-01,7.610846e-01,1.250000e+03
max,1.000000e+00,4.378282e+00,2.365287e+00,1.732370e+00,4.148023e+00,1.731978e+00,4.482618e+00,3.720345e+00,2.758590e+00,1.731450e+00,...,5.866367e+00,2.282217e+00,1.732740e+00,1.743123e+00,7.293420e+00,9.333287e+00,1.499064e+01,5.277313e+00,4.444690e+00,1.500000e+03


In [5]:
df["# label"].value_counts()

1.0    3500879
0.0    3499121
Name: # label, dtype: int64

The information found in the Pandas Profile report reveals the following.

* 7 Million rows of data
* 28 features and 1 target
* No missing data
* Our reponse, # label, is nearly balanced, close to 50%.
* We have 4 features in which there are only two values, which we could consider treating as binary/categorical.
  * f9
  * f13
  * f17
  * f21
* The remaing features are distrubuted normally or have a uniform distribution.

In [3]:
# Relabel target
df.rename(columns={'# label': 'target'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000000 entries, 0 to 6999999
Data columns (total 29 columns):
 #   Column  Dtype  
---  ------  -----  
 0   target  float64
 1   f0      float64
 2   f1      float64
 3   f2      float64
 4   f3      float64
 5   f4      float64
 6   f5      float64
 7   f6      float64
 8   f7      float64
 9   f8      float64
 10  f9      float64
 11  f10     float64
 12  f11     float64
 13  f12     float64
 14  f13     float64
 15  f14     float64
 16  f15     float64
 17  f16     float64
 18  f17     float64
 19  f18     float64
 20  f19     float64
 21  f20     float64
 22  f21     float64
 23  f22     float64
 24  f23     float64
 25  f24     float64
 26  f25     float64
 27  f26     float64
 28  mass    float64
dtypes: float64(29)
memory usage: 1.5 GB


# Modeling Preparations

*Which methods are you proposing to utilize to solve the problem?  Why is this method appropriate given the business objective? How will you determine if your approach is useful (or how will you differentiate which approach is more useful than another)?  More specifically, what evaluation metrics are most useful given that the problem is a classification one (ex., Accuracy, F1-score, Precision, Recall, AUC, etc.)?*


## Create training and test sets

In [4]:
df_small = df.sample(frac=0.1, random_state=1).copy(deep=True)

scaler = StandardScaler()

X = df_small.loc[:, df_small.columns != 'target'].values

y = df_small['target'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=30)

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
lr.fit(X_train, y_train)
print(lr.score(X_test, y_test))


0.8370735930735931


## Setup code for tensorboard use later

In [8]:
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

# Print current path
print(os.getcwd())
print(run_logdir)

tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)

d:\Projects\MSDS-7333-QTW
.\my_logs\run_2021_11_07-07_05_03


# Model Building & Evaluation

In this case, your primary task is to construct a neural network to detect the existence of new particles and will involve the following steps:

- Construct your neural network's architecture
- Fit your neural network to your training data
- Analyze your model's performance - referencing your chosen evaluation metric (including supplemental visuals and analysis where appropriate)"


## Define the model

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(.2, input_shape=(2,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(.2, input_shape=(2,)),
    tf.keras.layers.Dense(1, activation='linear')
])

## Compile the model and define metrics

In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

## Define an early stopping callback

This is a callback that will stop the training when the validation loss stops improving.

In [11]:
from tensorflow.keras.callbacks import EarlyStopping
safety= EarlyStopping(monitor='val_loss', patience=3)

## Fit the model

In [12]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), 
          callbacks=[tensorboard_cb, safety], batch_size=1000)

Epoch 1/10
4690/4690 [==============================] - 18s 4ms/step - loss: 0.4737 - accuracy: 0.7979 - val_loss: 0.3701 - val_accuracy: 0.8376
Epoch 2/10
4690/4690 [==============================] - 14s 3ms/step - loss: 0.4037 - accuracy: 0.8246 - val_loss: 0.3692 - val_accuracy: 0.8339
Epoch 3/10
4690/4690 [==============================] - 15s 3ms/step - loss: 0.3859 - accuracy: 0.8363 - val_loss: 0.3454 - val_accuracy: 0.8562
Epoch 4/10
4690/4690 [==============================] - 15s 3ms/step - loss: 0.3719 - accuracy: 0.8458 - val_loss: 0.3429 - val_accuracy: 0.8574
Epoch 5/10
4690/4690 [==============================] - 17s 4ms/step - loss: 0.3637 - accuracy: 0.8516 - val_loss: 0.3445 - val_accuracy: 0.8583
Epoch 6/10
4690/4690 [==============================] - 16s 3ms/step - loss: 0.3647 - accuracy: 0.8521 - val_loss: 0.3337 - val_accuracy: 0.8586
Epoch 7/10
4690/4690 [==============================] - 16s 3ms/step - loss: 0.3637 - accuracy: 0.8522 - val_loss: 0.3389 - val_ac

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 28)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               3712      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 20,353
Trainable params: 20,353
Non-trainable params: 0
____________________________________________________

In [14]:
model.evaluate(X_test, y_test, batch_size=1000)

2310/2310 [==============================] - 3s 1ms/step - loss: 0.3329 - accuracy: 0.8564


[0.3328903615474701, 0.8563519716262817]

## Tensorsboard

In [15]:
%load_ext tensorboard

In [6]:
from tensorflow.keras.callbacks import EarlyStopping

root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

# Print current path
print(os.getcwd())
print(run_logdir)

tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)

def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[28]):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
        model.add(tf.keras.layers.Dropout(.2, input_shape=(2,)))
    model.add(tf.keras.layers.Dense(1))
    #optimizer = 'adam'
    optimizer = tf.keras.optimizers.Adam()
    loss = tf.keras.losses.BinaryCrossentropy()
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    return model

keras_reg = tf.keras.wrappers.scikit_learn.KerasRegressor(build_model)

from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 500)
}

safety = EarlyStopping(monitor='val_loss', patience=3, min_delta=2e-4)

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=5, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=500,
                  validation_data=(X_test, y_test),
                  callbacks=[tensorboard_cb, safety], batch_size=100)

d:\Projects\MSDS-7333-QTW
.\my_logs\run_2021_11_10-16_56_54
Epoch 1/500
3127/3127 [==============================] - 13s 4ms/step - loss: 4.3317 - accuracy: 0.6185 - val_loss: 2.9395 - val_accuracy: 0.7228
Epoch 2/500
3127/3127 [==============================] - 10s 3ms/step - loss: 2.1555 - accuracy: 0.7691 - val_loss: 1.2522 - val_accuracy: 0.8022
Epoch 3/500
3127/3127 [==============================] - 9s 3ms/step - loss: 0.8516 - accuracy: 0.8039 - val_loss: 0.5937 - val_accuracy: 0.8002
Epoch 4/500
3127/3127 [==============================] - 10s 3ms/step - loss: 0.4861 - accuracy: 0.8170 - val_loss: 0.4373 - val_accuracy: 0.8268
Epoch 5/500
3127/3127 [==============================] - 10s 3ms/step - loss: 0.4537 - accuracy: 0.8153 - val_loss: 0.4225 - val_accuracy: 0.8308
Epoch 6/500
3127/3127 [==============================] - 10s 3ms/step - loss: 0.4261 - accuracy: 0.8287 - val_loss: 0.4233 - val_accuracy: 0.8306
Epoch 7/500
3127/3127 [==============================] - 10s 3ms/

RandomizedSearchCV(cv=3,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x0000022225C3E160>,
                   n_iter=5,
                   param_distributions={'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  5...
       404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416,
       417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429,
       430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442,
       443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455,
       456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468,
       469, 470, 471, 472, 47

In [7]:
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)

{'n_neurons': 17, 'n_hidden': 2}
-0.3490656912326813


In [13]:
import keras_tuner as kt

def build_model(hp):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(
      hp.Choice('units', [8, 16, 32]),
      activation='relu'))
  model.add(tf.keras.layers.Dropout(.2, input_shape=(2,)))  
  model.add(tf.keras.layers.Dense(1, activation='relu'))
  model.compile(loss=tf.keras.losses.BinaryCrossentropy())
  return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json


In [14]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test), batch_size=1000)
best_model = tuner.get_best_models()[0]

Trial 4 Complete [00h 00m 09s]
val_loss: 0.39247244596481323

Best val_loss So Far: 0.3811619281768799
Total elapsed time: 00h 00m 26s
INFO:tensorflow:Oracle triggered exit


In [17]:
print(best_model)

In [29]:
model = rnd_search_cv.best_estimator_.model
model.save('models/particle_random_cv')

INFO:tensorflow:Assets written to: models/particle_random_cv\assets
